In [1216]:
import pandas as pd
import json
from tqdm.autonotebook import tqdm

In [1217]:
df = pd.read_csv("data/AKG/Test Collection/AKGG/akg_standard_akgg_property_rele.csv")

In [35]:
import json

with open("data/AKG/Formal Run Topics/AKGG_Formal_Run_Topic.json") as json_file:
    data = json.load(json_file)
    qid, query, entity, entityType, action = [], [], [], [], []
    for p in data['queries']:
        qid.append(p['queryId'])
        query.append(p['query'])   
        entity.append(p['entity'])
        entityType.append(' '.join(p['entityTypes']))    
        action.append(p['action'])
topic = pd.DataFrame({"query_id": qid, "query": query, "entity": entity, "entityType": entityType, "action":action})

In [99]:
class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value

In [156]:

with open("data/AKG/Participants Runs/AKGG/akgg-formalrun-cuis.json") as json_file:
    data = json.load(json_file)
    run = AutoVivification()
    for p in data['runs']:
        for res in p['results']:
            for prop in res['properties']:
                run[p['runid']][str(res['queryid'])][str(prop['property'])] = prop['rank']


In [1223]:
df = df.merge(topic, how="inner", on="query_id")
df['query'] = df['query'].str.lower()
df['action'] = df['action'].str.lower()
df['entityType'] = df['entityType'].str.lower()
df['entity'] = df['entity'].str.lower()

In [367]:
import collections
qrel = collections.defaultdict(dict)
for qid, prop, label in df[['query_id', 'property', 'rele_label']].values:
    qrel[str(qid)][str(prop)] = int(label)
    

In [368]:
selected_qids = df[df['query'].isin(list(entity2wiki.keys()))].query_id.unique()

In [449]:
from pyNTCIREVAL import Labeler
from pyNTCIREVAL.metrics import MSnDCG, nERR, nDCG

res = []
for i in qrel:
    
    if int(i) not in selected_qids:
        continue
    # dict of { document ID: relevance level }
    qrels = qrel[str(i)] 
    grades = [1,2,3,4] # a grade for relevance levels 1 and 2 (Note that level 0 is excluded)
#     grades = list(set(qrels.values())) + [0]
    ranked_list = list(run['1'][str(i)].keys()) # a list of document IDs
#     ranked_list = naive[str(i)] # a list of document IDs
#     ranked_list =  {**naive[str(i)],  **run['1'][str(i)]}
#     for k in ranked_list:
#         ranked_list[k] *= -1

    # labeling: [doc_id] -> [(doc_id, rel_level)]
    labeler = Labeler(qrels)
    labeled_ranked_list = labeler.label(ranked_list)

    # compute the number of documents for each relevance level
    rel_level_num = 5
    xrelnum = labeler.compute_per_level_doc_num(rel_level_num)

    # Let's compute nDCG@5
    metric = MSnDCG(xrelnum, grades, cutoff=10)
    result = metric.compute(labeled_ranked_list)
    res.append(result)

print(np.mean(res))

0.5196837307322266


In [307]:
df_wiki = pd.read_csv("data/wikihowSep.csv")
df_wiki['headline'] = df_wiki['headline'].str.replace("\n", "")
df_wiki['title'] = df_wiki['title'].str.replace("How to", "")

In [787]:
df_wiki['overview'] = df_wiki['overview'].astype(str)
df_wiki['headline'] = df_wiki['headline'].astype(str)
df_wiki['text'] = df_wiki['text'].astype(str)
df_wiki['sectionLabel'] = df_wiki['sectionLabel'].astype(str)
df_wiki['title'] = df_wiki['title'].astype(str)

In [1322]:
df_wiki['overview'] = df_wiki['overview'].str.lower()
df_wiki['title'] = df_wiki['title'].str.lower()

In [340]:
%%time
queries = [q.lower() for q in df['query'].unique()]
query2wiki = collections.defaultdict(object)
for entity in tqdm(queries, total=len(queries)):
    df_ = df_wiki[df_wiki.title.str.contains(entity, na=False)]
    if len(df_) > 0:
        query2wiki[entity] = df_


CPU times: user 2min 11s, sys: 668 ms, total: 2min 12s
Wall time: 2min 12s


In [682]:
query2id = dict(zip(df['query'], df['query_id']))
query2entity = dict(zip(df['query'], df['entity']))
query2action = dict(zip(df['query'], df['action']))

In [456]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.
/Users/jarana/anaconda3/envs/keras/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jarana/anaconda3/envs/keras/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jarana/anaconda3/envs/keras/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/jarana/anacon

In [470]:
all_properties = []
for i in run["1"]:
    all_properties.extend(list(run["1"][i].keys()))

In [1231]:
matched_wiki = df_wiki[df_wiki.text.str.contains('|'.join(queries), na=False)]
wiki_all_text = []
for col in df_wiki.columns:
    wiki_all_text.extend(matched_wiki[col].unique())

In [1242]:
df.entityType.unique()

array(['thing action', 'thing creativework movie',
       'thing creativework musicrecording', 'thing creativework book',
       'thing creativework', 'thing creativework tvseries', 'thing event',
       'thing event sportsevent', 'thing event educationevent',
       'thing intangible', 'thing intangible computerlanguage',
       'thing intangible language', 'thing intangible service',
       'thing intangible service financialproduct',
       'thing medicalentity medicalprocedure',
       'thing medicalentity medicalintangible',
       'thing medicalentity medicalcondition',
       'thing medicalentity anatomicalsystem',
       'thing medicalentity substance drug',
       'thing medicalentity substance',
       'thing medicalentity parasiticdisease',
       'thing organization corporation',
       'thing organization sportsorganization',
       'thing organization broadcastchannel',
       'thing organization governmentorganization',
       'thing organization performinggroup musicgro

In [1245]:
df[['query_id', 'entity', 'entityType', 'action']].drop_duplicates().head()

,query_id,entity,entityType,action
0,1,benchmarking,thing action,have
11,2,bodybuilding,thing action,take
20,3,brainstorming,thing action,brainstorm
31,4,chain smoking,thing action,start
38,5,funding,thing action,get funding


In [1251]:
df[df.entityType == 'thing action']['property'].unique()

array(['startTime', 'result', 'target', 'name', 'actionStatus', 'object',
       'instrument', 'disambiguatingDescription', 'description',
       'participant', 'agent', 'potentialAction', 'endTime',
       'additionalType', 'image', 'location', 'alternateName', 'url',
       'identifier', 'error'], dtype=object)

In [1268]:
df[df.entityType == "thing product"].entity.nunique()

45

In [1264]:
df.groupby("entityType").size().sort_values(ascending=False)

entityType
thing product                                                     822
thing place                                                       434
thing creativework movie                                          379
thing person                                                      330
thing organization corporation                                    256
thing medicalentity medicalcondition                              200
thing event                                                       157
thing organization sportsorganization                             150
thing action                                                      139
thing product vehicle car                                         113
thing medicalentity anatomicalsystem                               82
thing creativework tvseries                                        61
thing intangible                                                   59
thing product vehicle                                              53
thing cre

[' winterize a jet ski',
 ' ski',
 ' clean jet',
 ' marshall a jet',
 ' slalom ski (water ski on one ski)']

In [1330]:
import re

regex = re.compile('[^a-zA-Z0-9]')
#First parameter is the replacement, second parameter is your input string
def proprocessingText(doc):
    doc = regex.sub(' ', doc)
    doc = " ".join([w for w in doc.split() if not w in stop_words])
    return doc

corpus = []
for idx, row in df_wiki[['overview', 'title']].drop_duplicates().iterrows():
    doc = proprocessingText(row['title'] + row['overview'])
    corpus.append(doc)

In [1331]:
from rank_bm25 import BM25Okapi

tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [1337]:
for idx, row in df[['query_id', 'entity', 'action']].drop_duplicates().iterrows():
    query = row['action'] + " " +row['entity']
    tokenized_query = query.split(" ")
    doc = bm25.get_top_n(tokenized_query, corpus, n=1)
    print(query)
    print(doc)
    break

have benchmarking
['identify prints3 digital photographic technology images transferred paper stone metal wood part good education art history involves studying identifying different printing processes printmaking field study life learn basics identifying relief intaglio planographic lithography start building identification skills']


In [505]:
topic_all_text = []
for col in ["property", "query", "entity", "entityType", "action"]:
    topic_all_text.extend(df[col].unique())

In [1197]:
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
raw_text = [str(i) for i in wiki_all_text + topic_all_text + all_properties]
# corpus = [w for w in " ".join(corpus).split() if not w in stop_words]

In [1169]:
import re

regex = re.compile('[^a-zA-Z0-9]')
#First parameter is the replacement, second parameter is your input string
def clean(doc):
    doc = regex.sub(' ', doc)
    doc = " ".join([w for w in doc.split() if not w in stop_words])
    return doc
#Out: 'abdE'
print((df_wiki.iloc[100].overview))
clean(df_wiki.iloc[100].overview)

 Stage names are used by all types of performers, from musicians, actors and roller derby athletes to burlesque dancers, belly dancers, and exotic dancers. A stage name can help a performer craft and reflect their public persona, or connect with the audience more effectively. A stage name can also help a performer maintain a separation between their public life and their personal life. 


'Stage names used types performers musicians actors roller derby athletes burlesque dancers belly dancers exotic dancers A stage name help performer craft reflect public persona connect audience effectively A stage name also help performer maintain separation public life personal life'

In [1173]:
import http.client, urllib.request, urllib.parse, urllib.error, base64
import json

def getVectors(queries):
    headers = {
        # Request headers
        'Content-Type': 'application/json',
        'Ocp-Apim-Subscription-Key': '924c1505854b4da4a6144a1cce92937f',
    }

    params = urllib.parse.urlencode({})
    
    try:
        conn = http.client.HTTPSConnection('api.msturing.org')
#         conn.request("POST", "/gen/encode?%s" % params, '{"queries": ["how to make gingerbread people (in grams)", "test AI"]}', headers)
        conn.request("POST", "/gen/encode?%s" % params, str({"queries": queries}).replace("\'", "\""), headers)
        response = conn.getresponse()
        data = response.read()
        data = json.loads(data)
        conn.close()
    except Exception as e:
        print(data)
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
    
    return {data[i]['query']:data[i]['vector'] for i in range(len(data))}

data = getVectors(list(df.property.unique())[:50])

In [1186]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([data["startTime"]], [data["time"]])

KeyError: 'time'

In [1150]:
import re

regex = re.compile('[^a-zA-Z0-9]')
#First parameter is the replacement, second parameter is your input string
regex.sub('', df_wiki.iloc[0].overview)
#Out: 'abdE'

corpus = []
for t in raw_text:
    corpus.append(" ".join([w for w in t.split() if not w in stop_words]))

In [1199]:
corpus = raw_text

In [1200]:
MAX_NUM_WORDS = 100000
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(corpus)

In [1201]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 34795 unique tokens.


In [1211]:
def get_pretrain_embeddings(path, word_index, EMBEDDING_DIM=300):
    MAX_NUM_WORDS = len(word_index)
    BASE_DIR = path + 'data/'
    GLOVE_DIR = os.path.join(BASE_DIR, 'w2v')
    print('Indexing word vectors.')

    embeddings_index = {}
    with open(os.path.join(GLOVE_DIR, 'glove.42B.300d.txt'), encoding="utf-8") as f:
        for line in f:
            word, coefs = line.split(maxsplit=1)
            coefs = np.fromstring(coefs, 'f', sep=' ')
            embeddings_index[word] = coefs

    print('Found %s word vectors.' % len(embeddings_index))

    print('Preparing embedding matrix.')

    # prepare embedding matrix
    num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
    embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
    found = 0
    for word, i in word_index.items():
        if i > MAX_NUM_WORDS:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if embedding_vector.shape[0] == 0:
                continue
            embedding_matrix[i] = embedding_vector
            found += 1

    print("Token num: %d, Found Tokens: %d" % (len(word_index), found))

    # load pre-trained word embeddings into an Embedding layer
    embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                embeddings_initializer=Constant(embedding_matrix))

    return embedding_layer

In [1212]:
from Models import getDSSM, getRanker

path = "/Users/jarana/workspace/WikiHow-Task-Based/"
max_words = len(word_index)
embedding_layer = get_pretrain_embeddings(path, word_index)

Indexing word vectors.
Found 1917494 word vectors.
Preparing embedding matrix.
Token num: 34795, Found Tokens: 29582


In [532]:
from keras.models import Sequential
from keras import Input, Model
from keras.layers import GlobalMaxPooling1D, Dot, LSTM, Dense, Concatenate


def getDSSM(embedding_layer, MAX_SEQUENCE_LENGTH):

    q_inp = Input(shape=(MAX_SEQUENCE_LENGTH,))
    t_inp = Input(shape=(MAX_SEQUENCE_LENGTH,))
#     lstm = LSTM(100)
    q_emb = GlobalMaxPooling1D()(embedding_layer(q_inp))
    t_emb = GlobalMaxPooling1D()(embedding_layer(t_inp))
#     q_emb = lstm(embedding_layer(q_inp))
#     t_emb = lstm(embedding_layer(t_inp))
    pred = Dot(-1, normalize=True)([q_emb, t_emb])
    model = Model([q_inp, t_inp], pred)

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model
model = getDSSM(embedding_layer, MAX_SEQUENCE_LENGTH)

In [ ]:
x_task = pad_sequences(tokenizer.texts_to_sequences(data["t"].tolist()), maxlen=MAX_SEQUENCE_LENGTH)
x_query1 = pad_sequences(tokenizer.texts_to_sequences(data["q1"].tolist()), maxlen=MAX_SEQUENCE_LENGTH)
x_query2 = pad_sequences(tokenizer.texts_to_sequences(data["q2"].tolist()), maxlen=MAX_SEQUENCE_LENGTH)
y = np.array([[i,j] for i,j in zip(data.label1.values, data.label2.values)])

In [595]:
run["1"]

{'1': {'result': 1,
  'target': 2,
  'participant': 3,
  'identifier': 4,
  'location': 5,
  'instrument': 6,
  'name': 7,
  'object': 8,
  'error': 9,
  'image': 10,
  'description': 11,
  'agent': 12,
  'url': 13,
  'additionalType': 14,
  'startTime': 15,
  'potentialAction': 16,
  'endTime': 17,
  'actionStatus': 18,
  'alternateName': 19},
 '2': {'name': -1,
  'participant': -2,
  'result': -3,
  'location': -4,
  'target': -5,
  'object': -6,
  'agent': -7,
  'image': -8,
  'identifier': -9,
  'error': -10,
  'instrument': -11,
  'description': -12,
  'url': -13,
  'startTime': -14,
  'endTime': -15,
  'alternateName': -16,
  'additionalType': -17,
  'actionStatus': -18,
  'potentialAction': -19},
 '3': {'participant': 1,
  'location': 2,
  'name': 3,
  'object': 4,
  'result': 5,
  'target': 6,
  'identifier': 7,
  'instrument': 8,
  'image': 9,
  'error': 10,
  'agent': 11,
  'description': 12,
  'url': 13,
  'startTime': 14,
  'endTime': 15,
  'potentialAction': 16,
  'additio

In [ ]:
from pyNTCIREVAL import Labeler
from pyNTCIREVAL.metrics import MSnDCG, nERR, nDCG

def evaluate(qrels, ranked_list):
    res = []
    grades = [1,2,3,4] # a grade for relevance levels 1 and 2 (Note that level 0 is excluded)
    labeler = Labeler(qrels)
    labeled_ranked_list = labeler.label(ranked_list)
    rel_level_num = 5
    xrelnum = labeler.compute_per_level_doc_num(rel_level_num)
    metric = MSnDCG(xrelnum, grades, cutoff=10)
    result = metric.compute(labeled_ranked_list)
    return result

[]

In [705]:
for col in df_wiki.columns:
    res = []
    for query in query2wiki:
        qrels = qrel[str(query2id[query])]
#         cand_properties = list(run['1'][str(query2id[query])].keys())
        cand_properties = list(run['1'][str(query2id[query])].keys()) + list(qrels.keys())
        entity = str(query2entity[query])
        action = str(query2action[query])
        wikis = [str(i) for i in query2wiki[query][col].unique()]
        x_wikis = pad_sequences(tokenizer.texts_to_sequences(wikis), maxlen=MAX_SEQUENCE_LENGTH)

        rank = {}
        for prop in cand_properties:
            x_props = pad_sequences(tokenizer.texts_to_sequences([entity+ " " + action + " " + prop] * len(x_wikis)), maxlen=MAX_SEQUENCE_LENGTH)
            rank[prop] = np.max(model.predict([x_props, x_wikis]))
        rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)]
        res.append(evaluate(qrels, rank))
    

    print(col, np.mean(res))

overview 0.4878393658549674
headline 0.5275873391574725
text 0.4987795913292341
sectionLabel 0.5243990747826948
title 0.5345313329574838


In [650]:
len(df), df.query_id.nunique(), df.property.nunique(), df.entityType.nunique(), df['query'].nunique()

(3796, 229, 296, 41, 229)

In [676]:
trainIds, testIds = [], []
for name, group in df.groupby("entityType"):
    if group.query_id.nunique() > 1:
        ids = list(group.query_id.unique())
        mid = int(group.query_id.nunique() / 2)
        trainIds.extend(ids[:mid])
        testIds.extend(ids[mid:])
    else:
        ids = list(group.query_id.unique())
        trainIds.extend(ids)

In [677]:
df_train = df[df.query_id.isin(trainIds)]
df_test = df[df.query_id.isin(testIds)]

In [678]:
len(df_test), len(df_train)

(1857, 1939)

In [680]:
len(df)

3796

In [979]:
from keras.models import Sequential
from keras import Input, Model
from keras.layers import GlobalMaxPooling1D, Dot, LSTM, Dense, Concatenate


def getPropModel(embedding_layer, PROPERTY_NUN, MAX_QUERY_SEQUENCE_LENGTH, MAX_DOC_SEQUENCE_LENGTH):

    q_inp = Input(shape=(MAX_QUERY_SEQUENCE_LENGTH,))
    d_inp = Input(shape=(MAX_DOC_SEQUENCE_LENGTH,))
    
    q_lstm = LSTM(100)
    d_lstm = LSTM(100)
#     q_emb = q_lstm(embedding_layer(q_inp))
#     d_emb = d_lstm(embedding_layer(d_inp))
    q_emb = GlobalMaxPooling1D()(embedding_layer(q_inp))
    d_emb = GlobalMaxPooling1D()(embedding_layer(d_inp))
    concat = Concatenate()([q_emb, d_emb])
    dense = Dense(PROPERTY_NUN, activation='sigmoid')
    pred = dense(concat)
    model = Model([q_inp, d_inp], pred)

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

for col in df_wiki.columns:
    PROPERTY_NUN = df.property.nunique() 
    MAX_QUERY_SEQUENCE_LENGTH = np.max([len(i.split()) for i in df.entity.unique()]) + np.max([len(i.split()) for i in df.action.unique()])
    MAX_DOC_SEQUENCE_LENGTH = np.max([len(str(i).split()) for i in df_wiki[col].unique()])
    model = getPropModel(embedding_layer, PROPERTY_NUN, MAX_QUERY_SEQUENCE_LENGTH, MAX_DOC_SEQUENCE_LENGTH)
    
    break
#     for qid in trainIds:
#         label = qrel[qid].keys()
        
#         entity = str(query2entity[query])
#         action = str(query2action[query])
        

In [980]:
from keras.utils import to_categorical

In [981]:
prop2id = {j:i for i, j in zip(range(df.property.nunique()), df.property.unique())}
id2prop = {i:j for i, j in zip(range(df.property.nunique()), df.property.unique())}

In [982]:
PROPERTY_NUN = df.property.nunique() 


In [1024]:
df_wiki.head()

,overview,headline,text,sectionLabel,title
0,So you're a new or aspiring artist and your c...,Sell yourself first.,"Before doing anything else, stop and sum up y...",Steps,sell fine art online
1,"If you want to be well-read, then, in the wor...",Read the classics before 1600.,Reading the classics is the very first thing ...,Reading the Classics,be well read
2,So you're a new or aspiring artist and your c...,Join online artist communities.,Depending on what scale you intend to sell yo...,Steps,sell fine art online
3,So you're a new or aspiring artist and your c...,Make yourself public.,Get yourself out there as best as you can by ...,Steps,sell fine art online
4,So you're a new or aspiring artist and your c...,Blog about your artwork.,"Given the hundreds of free blogging websites,...",Steps,sell fine art online


In [1079]:
# for col in ['overview', 'headline', 'text', 'sectionLabel', 'title']:
for col in ['overview']:
    MAX_DOC_SEQUENCE_LENGTH = int(np.mean([len(str(i).split()) for i in df_wiki[col].unique()]))
    res = []
    train_id, train_query, train_doc, train_y = [], [], [], []
    test_id, test_query, test_doc, test_y = [], [], [], []
    
    for key in query2wiki:
        qid = query2id[key]
        qrels = qrel[str(qid)]
#       convert prop to prop_id
        label = to_categorical([prop2id[i] for i in list(qrels.keys())], num_classes=PROPERTY_NUN)
        label = np.sum(label, axis=0)
        
        entity = str(query2entity[key])
        action = str(query2action[key])
        
        for wiki in query2wiki[key][col].unique():
            query = pad_sequences(tokenizer.texts_to_sequences([entity + " " + action] * len(wikis)), maxlen=MAX_QUERY_SEQUENCE_LENGTH)[0]
            
            doc = pad_sequences(tokenizer.texts_to_sequences([str(wiki)]), maxlen=MAX_DOC_SEQUENCE_LENGTH)[0]
            train_query.append(query) if qid in trainIds else test_query.append(query)
            train_doc.append(doc) if qid in trainIds else test_doc.append(doc)
            train_y.append(label) if qid in trainIds else test_y.append(label)
            train_id.append(qid) if qid in trainIds else test_id.append(qid)
#         rank = {}
#         for prop in cand_properties:
#             x_props = pad_sequences(tokenizer.texts_to_sequences([entity+ " " + action + " " + prop] * len(x_wikis)), maxlen=MAX_SEQUENCE_LENGTH)
#             rank[prop] = np.max(model.predict([x_props, x_wikis]))
#         rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)]
#         res.append(evaluate(qrels, rank))
    train_query = np.array(train_query)
    train_doc = np.array(train_doc)
    train_y = np.array(train_y)
    test_query = np.array(test_query)
    test_doc = np.array(test_doc)
    test_y = np.array(test_y)
    break


In [1080]:
model = getPropModel(embedding_layer, PROPERTY_NUN, MAX_QUERY_SEQUENCE_LENGTH, MAX_DOC_SEQUENCE_LENGTH)


In [1083]:
for i in range(20):
    model.fit([train_query, train_doc], train_y, validation_data=([test_query, test_doc], test_y), verbose=1)
    eval_model(model)

Train on 923 samples, validate on 1074 samples
Epoch 1/1
923/923 [==============================] - 10s 10ms/step - loss: 0.2130 - acc: 0.9132 - val_loss: 0.1494 - val_acc: 0.9471
overview 0.34589413846440814
Train on 923 samples, validate on 1074 samples
Epoch 1/1
923/923 [==============================] - 4s 4ms/step - loss: 0.0624 - acc: 0.9841 - val_loss: 0.1522 - val_acc: 0.9459
overview 0.3645246373432326
Train on 923 samples, validate on 1074 samples
Epoch 1/1
923/923 [==============================] - 4s 4ms/step - loss: 0.0363 - acc: 0.9929 - val_loss: 0.1512 - val_acc: 0.9446
overview 0.35729992402887867
Train on 923 samples, validate on 1074 samples
Epoch 1/1
923/923 [==============================] - 4s 4ms/step - loss: 0.0251 - acc: 0.9960 - val_loss: 0.1528 - val_acc: 0.9453
overview 0.3649630336741482
Train on 923 samples, validate on 1074 samples
Epoch 1/1
923/923 [==============================] - 4s 4ms/step - loss: 0.0192 - acc: 0.9971 - val_loss: 0.1542 - val_acc: 0

KeyboardInterrupt: 

In [1018]:
for col in df_wiki.columns:
    res = []
    for query in query2wiki:
        qid = query2id[query]
        if qid not in test_id:
            continue
        qrels = qrel[str(query2id[query])]
        cand_properties = list(run['1'][str(query2id[query])].keys())
        res.append(evaluate(qrels, cand_properties))
    print(col, np.mean(res))    
    break
    



overview 0.5134289962516724


In [1038]:
def eval_model(model):
    pred = model.predict([test_query, test_doc])  
    sort_pred = np.argsort(pred)[:,::-1][:,:20]
    runLSTM = {}
    for i, j in zip(test_id, sort_pred):
        runLSTM[i] = [id2prop[k] for k in j]
    for col in df_wiki.columns:
        res = []
        for query in query2wiki:
            qid = query2id[query]
            if qid not in test_id:
                continue
            qrels = qrel[str(query2id[query])]
            cand_properties = runLSTM[qid]
            res.append(evaluate(qrels, cand_properties))
        print(col, np.mean(res))    
        break
    




In [1088]:
model.predict([test_query, test_doc])  


array([[0.00647825, 0.00362948, 0.00058389, ..., 0.00149783, 0.00060281,
        0.00103131],
       [0.00709313, 0.00621337, 0.00071162, ..., 0.00249559, 0.00092068,
        0.00189939],
       [0.00518009, 0.0054366 , 0.00166786, ..., 0.00283179, 0.00103575,
        0.00197542],
       ...,
       [0.00811243, 0.01550895, 0.00199294, ..., 0.00211614, 0.00071183,
        0.0018419 ],
       [0.00306621, 0.00964779, 0.00037026, ..., 0.00046399, 0.00017369,
        0.00075141],
       [0.00938502, 0.01903677, 0.00328699, ..., 0.00338045, 0.00276545,
        0.00454223]], dtype=float32)

In [1071]:
train_doc[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0], dtype=int32)